In [1]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

In [2]:
from astropy.utils import iers
iers.conf.auto_download = False

In [3]:
d = salobj.Domain()

In [4]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=1)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

In [5]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

[None, None, None, None, None, None, None]

AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 10 messages


# ATMCS

In [21]:
await asyncio.sleep(5.)
print("Done")
await salobj.set_summary_state(atmcs, salobj.State.ENABLED)

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 72 messages
falling behind; queue contains 71 elements
falling behind; queue contains 70 elements
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 22 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 22 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 22 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 53 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 22 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 22 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 22 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements


Done


[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

# ATPtg

In [22]:
await atptg.tel_timeAndDate.next(flush=True, timeout=5)
await salobj.set_summary_state(atptg, salobj.State.ENABLED)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [23]:
await atptg.cmd_azElTarget.set_start(azDegs=243., elDegs=80.)

RemoteTelemetry(ATDome, 1, position) falling behind; read 15 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 15 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 15 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 15 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 15 messages


In [7]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [10]:
ra = Angle("22:18:30.0947834", unit=u.hour)
dec = Angle("-60:15:34.526307", unit=u.deg)
target_name="Alpha Tuc"
radec = ICRS(ra, dec)

In [13]:
await atmcs.cmd_startTracking.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 85 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 85 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 85 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling be

In [17]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

RemoteTelemetry(ATDome, 1, position) falling behind; read 26 messages
falling behind; queue contains 11 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 27 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 26 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 27 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 27 messages


58738.19837464163
slew...


In [ ]:
await asyncio.gather(atmcs.close(), 
                     atptg.close(),
                     ataos.close(),
                     atpne.close(),
                     athex.close(),
                     atdome.close(),
                     atdomtraj.close())